## Preliminaries

In [1]:
import convokit

In [2]:
ROOT_DIR = '/kitchen/convokit_corpora/tennis-corpus/'

In [3]:
corpus = convokit.Corpus(ROOT_DIR)
corpus.load_info('utterance',['arcs_censored', 'question_motifs', 'question_motifs__sink'])

In [4]:
VERBOSITY = 10000

## Using PromptTypes to get types of questions

In [5]:
def question_filter(utt, aux_input={}):
    return utt.meta['is_question']
def response_filter(utt, aux_input={}):
    return (not utt.meta['is_question']) and (utt.reply_to is not None)

In [6]:
from convokit.prompt_types import PromptTypes

In [7]:
pt = PromptTypes(prompt_field='question_motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='question_motifs__sink',
                 output_field='prompt_types', prompt__tfidf_min_df=50,
                 ref__tfidf_min_df=50, 
    random_state=1000, verbosity=1)

In [8]:
pt.fit(corpus)

fitting 64571 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 8 prompt types


In [9]:
test_utt_id = '5188_0.q'
utt = corpus.get_utterance(test_utt_id)

In [10]:
utt.text

'How do you feel? Watching the Australian Open it was very scary watching your ankle buckle. How does your ankle feel now?'

In [11]:
utt.get_info('question_motifs__sink')

['feel_*__feel_do__how>*',
 'feel_*__feel_does__feel_how feel_*__feel_now__how>*']

### Per-utterance behavior

In [12]:
utt = pt.transform_utterance(utt)

In [13]:
utt.get_info('prompt_types__prompt_repr')

[0.04614480560093584,
 0.12523176450740134,
 -0.054075221364537254,
 -0.16488221294276534,
 -0.2622154385109799,
 0.06781246550773161,
 0.3161285586302379,
 -0.11348710370703316,
 -0.30383347022247026,
 -0.36553848329504285,
 0.3362904355819349,
 0.048598494761011045,
 -0.2667895757175696,
 -0.14162106717317796,
 0.45694706042159894,
 -0.09758869692105782,
 0.06397073233732087,
 -0.04206323851119204,
 0.07013280562733451,
 0.007080600454132783,
 0.1910828848593811,
 -0.15056265464413737,
 0.21175490736928645,
 0.010463721267544844]

In [18]:
utt.get_info('prompt_types__prompt_dists.8')

[1.1542767147522681,
 1.1836322878230092,
 1.1403839094093262,
 1.080862067095506,
 1.3137305097357392,
 0.36497031797843443,
 1.3267111408021754,
 1.1126782406483375]

In [19]:
utt.get_info('prompt_types__prompt_type.8')

5.0

In [20]:
utt.get_info('prompt_types__prompt_type_dist.8')

0.36497031797843443

### model output

In [49]:
for i in range(8):
    print(i)
    pt.display_type(i, corpus=corpus, k=5)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
try_*,0.664072,1.090674,1.319580,1.249188,1.087935,1.363645,1.155572,1.191221,0.0
how>*__keep_*__keep_how,0.669997,1.136478,1.267361,1.099178,1.043041,1.360519,1.171606,1.098759,0.0
do_*__do_do__do_what,0.686736,0.887752,1.325944,1.289076,1.022363,1.353739,1.343151,1.068488,0.0
do_*__do_do,0.690401,0.978398,1.269106,1.202296,0.977706,1.297059,1.308501,1.003302,0.0
prepare_*__prepare_for,0.692725,0.925045,1.307146,1.273835,1.118497,1.314042,1.307853,1.204177,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
work_for,0.690725,1.043187,1.391620,1.146133,1.049347,1.270692,1.245960,1.159645,0.0
work_*,0.693785,1.098218,1.385079,1.114691,1.074291,1.277762,1.233976,1.176795,0.0
control_*,0.697097,1.072021,1.325468,1.252930,1.098485,1.391434,1.204093,1.116907,0.0
focus_on,0.709495,0.960281,1.259950,1.186321,1.000408,1.268750,1.241740,1.216111,0.0
focus_*,0.710258,0.970667,1.266089,1.192621,1.004985,1.265143,1.228822,1.213640,0.0


top prompts:
2020_8.q On a physical level, you're stronger maybe than ever. How do you work hard if the tournament is so close? What do you do in the gym? How do you keep the regimen going on?
['how>*__how>do work_*', 'do_*__do_in__do_what__what>* do_*__do_what__what>*__what>do', 'how>*__how>do how>*__keep_*__keep_how keep_*__keep_do']

887_3.q When you're in command of a match like that, do you actually try different things or work on something?
['try_*__try_do when>* work_*']

4182_6.q In the beginning of this season you lost in Melbourne, the final. Did you try to make a difference in preparation before today's game and why it didn't work?
['did>* try_* work_*']

3180_28.q How do you do that? What do you do?
['do_*__do_do do_*__do_how how>*__how>do', 'do_*__do_what__what>*__what>do']

5541_2.q What do you do between now and Wimbledon? What things do you work on?
['do_*__do_what__what>*__what>do', 'what>* work_*']

top responses:
3881_6.a No, it's me definitely. I'm really trying. Th

...

top prompts:
1226_7.q Youzhny is a player with a lot of big wins.  He's won against Nadal, won against Federer.  Was it a sufficient match, and what was the approach for you before you went into the game?
['was>* was_*__was_what']

3419_4.q You somehow changed your game towards the end. Was it what your coach that told you to change it?
['was>* was_*__was_what']

3726_12.q Then your reaction at the end of the match, what was your feeling? Was it happy? Relief? A little bit of unhappiness because you almost let it go?
['was_*__was_what', 'was>*', '', '']

1984_8.q Was it mental or what was it?
['was>* was_*__was_what']

5869_5.q How much were you following the US Open at all? What was your reaction to seeing Marin and Kei in the final? Was the result surprising to you?
['how>*', 'was_*__was_what__what>*', 'was>*']

top responses:
2850_8.a Actually, September 1st was the first day back to school, and I missed it (smiling). But it started already. So my friends attended some classes alrea

,0,1,2,3,4,5,6,7,type_id
do>*__think_'s__think_*,1.155789,0.996740,1.013655,1.067685,1.062727,1.260903,1.513347,0.531644,7.0
is>*,1.062490,1.067104,1.080466,1.062832,1.071693,1.189330,1.466067,0.538167,7.0
think_'s__think_*,1.169565,0.959485,1.010322,1.085590,1.038018,1.252852,1.516804,0.552625,7.0
like_*__like_do,1.071264,0.986405,1.009678,1.153968,1.063828,1.215064,1.495786,0.581972,7.0
is_*__is_like,1.128784,1.161261,1.067291,0.934017,1.014746,1.132462,1.387028,0.589775,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
think_is,1.129770,1.083397,0.934999,1.095067,1.139335,1.220646,1.479282,0.544207,7.0
is_there,1.125157,1.102398,1.057348,1.093204,1.092229,1.258255,1.426755,0.552333,7.0
think_are,1.147441,1.094366,0.939655,1.069253,1.088532,1.184831,1.462403,0.561257,7.0
think_'s,1.135482,1.098581,0.953449,1.067127,1.089396,1.259378,1.460707,0.574519,7.0
's_about,1.088023,1.117607,1.035078,1.076636,1.064573,1.207458,1.427095,0.577848,7.0


top prompts:
6218_1.q Do you think it's just confidence? Is that something that confidence can do in those big moments?
["do>*__think_'s__think_*", 'is>*']

1802_3.q It's obviously very entertaining to watch slices, lobs and dropshots. Do you think it's possible to do that against some of the players you might face later? Is that just a one-off today?
["do>*__think_'s__think_*", 'is>*']

1389_5.q The crowd likes you very much because not only do you play great tennis, but also you have the quality of being able to have fun and joke on the court. But now you seem more calm. Is it because you really want to focus and be the best in tennis? Do you think it's sort of a better discipline?
['is>*', "do>*__think_'s__think_*"]

5447_2.q I know that injuries are a personal thing. There's been a big trend of people pulling out today. Do you think it's something about the surface? Is this all just a coincidence?
["do>*__think_'s__think_*", 'is>*']

4898_10.q Do you think it's about patience durin

### transforming the entire corpus

In [21]:
corpus = pt.transform(corpus)

In [22]:
utt1 = corpus.get_utterance('2020_8.q')

In [23]:
utt1.text

"On a physical level, you're stronger maybe than ever. How do you work hard if the tournament is so close? What do you do in the gym? How do you keep the regimen going on?"

In [24]:
utt1.get_info('prompt_types__prompt_type.8')

0.0

## Storing models

In [25]:
import os

In [26]:
pt.dump_model(os.path.join(ROOT_DIR, 'pt_model'))

dumping embedding model
dumping training embeddings
dumping type model 8


In [27]:
new_pt = PromptTypes(prompt_field='question_motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='question_motifs__sink',
                 output_field='prompt_types_new', prompt__tfidf_min_df=50,
                 ref__tfidf_min_df=50, 
    random_state=1000, verbosity=1)

In [28]:
pt_model_dir = os.path.join(ROOT_DIR, 'pt_model')
!ls $pt_model_dir

km_model.8.joblib	   svd_model.joblib	   train_ref_ids.npy
prompt_df.8.tsv		   train_prompt_df.8.tsv   train_ref_vects.npy
prompt_tfidf_model.joblib  train_prompt_ids.npy    U_prompt.npy
ref_df.8.tsv		   train_prompt_vects.npy  U_ref.npy
ref_tfidf_model.joblib	   train_ref_df.8.tsv


In [29]:
new_pt.load_model(pt_model_dir)

loading embedding model
loading training embeddings
loading type model 8


In [30]:
utt = new_pt.transform_utterance(utt)

In [32]:
utt.get_info('prompt_types_new__prompt_type.8')

5.0

## Changing the number of types

In [35]:
pt.refit_types(4)

fitting 4 prompt types


In [47]:
for i in range(4):
    print(i)
    pt.display_type(i, type_key=4, k=5)
    print('\n\n')

0
top prompt:


,0,1,2,3,type_id
will>*,0.630950,1.298031,1.065377,1.418889,0.0
are>*__going_*,0.679646,1.262736,1.145024,1.406238,0.0
are_*__are_what,0.686385,1.343151,0.991287,1.467312,0.0
going_*__going_are,0.690053,1.251750,1.158506,1.423621,0.0
going_*,0.706212,1.269886,1.170854,1.368935,0.0


top response:


,0,1,2,3,type_id
take_will,0.725783,1.161619,1.122077,1.345260,0.0
know_yet,0.739357,1.295608,1.023017,1.251791,0.0
have_'ll,0.749098,1.289240,0.960487,1.480158,0.0
will_*,0.785161,1.262358,1.010793,1.288739,0.0
have_hopefully,0.786720,1.110688,1.054252,1.460314,0.0





1
top prompt:


...

,0,1,2,3,type_id
think_is,1.060018,1.142320,0.611579,1.409632,2.0
think_are,1.066465,1.114231,0.612049,1.389874,2.0
think_'s,1.060338,1.174667,0.629274,1.386505,2.0
is_there,1.073984,1.184221,0.643225,1.359634,2.0
think_now,0.976245,1.106304,0.644016,1.384676,2.0





3
top prompt:


,0,1,2,3,type_id
was_*__was_what,1.295417,1.241937,1.300852,0.428956,3.0
was_*__what>*,1.304098,1.228490,1.314929,0.437765,3.0
was_*__was_what__what>*,1.297477,1.230933,1.312691,0.439647,3.0
were_*,1.288189,1.243271,1.277479,0.440140,3.0
was_*,1.335885,1.213531,1.286668,0.445144,3.0


top response:


,0,1,2,3,type_id
was_actually,1.290001,1.245703,1.251458,0.445544,3.0
was_just,1.308849,1.208106,1.294344,0.471725,3.0
took_so,1.248846,1.235757,1.286594,0.484393,3.0
gave_*,1.313197,1.169273,1.330380,0.488466,3.0
was_surprised,1.299895,1.245011,1.271755,0.492075,3.0


## Example variation: using arcs instead of motifs

In [44]:
pt_arcs = PromptTypes(prompt_field='arcs_censored', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='arcs_censored',
                 output_field='prompt_types_arcs', prompt__tfidf_min_df=50,
                 ref__tfidf_min_df=50, n_types=4,
    random_state=1000, verbosity=1)

In [45]:
pt_arcs.fit(corpus)

fitting 78215 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 4 prompt types


In [46]:
for i in range(4):
    print(i)
    pt_arcs.display_type(i,  k=5)
    print('\n\n')

0
top prompt:


,0,1,2,3,type_id
feel_does,0.665389,0.965880,1.331548,1.141363,0.0
feel_do,0.674004,1.103788,1.349711,1.085030,0.0
feel_now,0.702311,1.164617,1.280405,1.125567,0.0
feel_*,0.709954,1.191471,1.180174,1.217915,0.0
feeling_now,0.720253,1.154975,1.247254,1.152868,0.0


top response:


,0,1,2,3,type_id
'm_fit,0.656866,1.124501,1.303152,1.183858,0.0
feels_good,0.675811,1.071909,1.271421,1.159534,0.0
feels_*,0.685173,1.041319,1.307112,1.151629,0.0
feels_yeah,0.693127,1.113668,1.293664,1.172959,0.0
feel_been,0.695478,1.137340,1.295989,1.192680,0.0





1
top prompt:


...

,0,1,2,3,type_id
was_actually,1.180324,1.244486,0.464286,1.354411,2.0
was_yeah,1.192064,1.305419,0.478270,1.361170,2.0
was_just,1.157896,1.277145,0.481335,1.366745,2.0
was_really,1.156121,1.291358,0.483771,1.370890,2.0
was_there,1.161162,1.242514,0.486821,1.382533,2.0





3
top prompt:


,0,1,2,3,type_id
are_what,1.232947,1.039911,1.451706,0.653125,3.0
expect_do,1.152844,1.033127,1.461430,0.659696,3.0
will>*,1.197333,1.060435,1.453046,0.663972,3.0
be_will,1.172330,1.108597,1.459242,0.682239,3.0
going_'re,1.149314,1.054815,1.458452,0.697464,3.0


top response:


,0,1,2,3,type_id
see_'ll,1.100716,1.058801,1.395763,0.718711,3.0
have_'ll,1.187852,0.978814,1.476159,0.729195,3.0
see_happens,1.135575,1.113555,1.375629,0.743777,3.0
see_will,1.091617,1.055427,1.348109,0.745719,3.0
see_goes,1.092629,1.105348,1.381137,0.746129,3.0


In [54]:
df = pt.train_types[8]['prompt_df']

In [55]:
df[]

 7.0    17273
 6.0    13076
 1.0     8328
-1.0     7765
 4.0     5791
 5.0     4379
 2.0     4213
 3.0     1775
 0.0     1567
Name: type_id, dtype: int64

In [56]:
pt.display_type(7, corpus=corpus, k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
do>*__think_'s__think_*,1.155789,0.996740,1.013655,1.067685,1.062727,1.260903,1.513347,0.531644,7.0
is>*,1.062490,1.067104,1.080466,1.062832,1.071693,1.189330,1.466067,0.538167,7.0
think_'s__think_*,1.169565,0.959485,1.010322,1.085590,1.038018,1.252852,1.516804,0.552625,7.0
like_*__like_do,1.071264,0.986405,1.009678,1.153968,1.063828,1.215064,1.495786,0.581972,7.0
is_*__is_like,1.128784,1.161261,1.067291,0.934017,1.014746,1.132462,1.387028,0.589775,7.0
like_*__like_do__like_what,1.158260,1.085909,1.073930,1.106000,1.081123,1.172475,1.445129,0.596329,7.0
think_*__think_is__think_why,1.106006,1.153582,1.005937,1.060637,1.031982,1.275838,1.354624,0.596931,7.0
like_*__what>*__what>do,1.067488,1.098119,1.102917,1.078895,1.039799,1.149338,1.420797,0.602374,7.0
's_*,1.058483,0.873674,1.098032,1.079070,0.990816,1.222524,1.540646,0.610405,7.0
like_*__like_do__what>*,1.079468,1.042654,1.095969,1.105133,1.049147,1.168217,1.453664,0.610522,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
think_is,1.129770,1.083397,0.934999,1.095067,1.139335,1.220646,1.479282,0.544207,7.0
is_there,1.125157,1.102398,1.057348,1.093204,1.092229,1.258255,1.426755,0.552333,7.0
think_are,1.147441,1.094366,0.939655,1.069253,1.088532,1.184831,1.462403,0.561257,7.0
think_'s,1.135482,1.098581,0.953449,1.067127,1.089396,1.259378,1.460707,0.574519,7.0
's_about,1.088023,1.117607,1.035078,1.076636,1.064573,1.207458,1.427095,0.577848,7.0
think_always,1.160450,1.146366,1.056997,1.030997,1.148802,1.090964,1.443062,0.579397,7.0
's_really,1.102912,1.138633,1.041846,1.073410,1.062061,1.272286,1.408632,0.579524,7.0
are_*,1.150882,1.140159,0.990419,1.088843,1.123055,1.227889,1.412462,0.584224,7.0
's_for,1.174044,1.151047,0.933325,1.103528,1.145505,1.252021,1.403483,0.584919,7.0
's_have,1.134782,1.183457,1.011616,1.062724,1.103204,1.238154,1.378383,0.584923,7.0


top prompts:
6218_1.q Do you think it's just confidence? Is that something that confidence can do in those big moments?
["do>*__think_'s__think_*", 'is>*']

1802_3.q It's obviously very entertaining to watch slices, lobs and dropshots. Do you think it's possible to do that against some of the players you might face later? Is that just a one-off today?
["do>*__think_'s__think_*", 'is>*']

1389_5.q The crowd likes you very much because not only do you play great tennis, but also you have the quality of being able to have fun and joke on the court. But now you seem more calm. Is it because you really want to focus and be the best in tennis? Do you think it's sort of a better discipline?
['is>*', "do>*__think_'s__think_*"]

5447_2.q I know that injuries are a personal thing. There's been a big trend of people pulling out today. Do you think it's something about the surface? Is this all just a coincidence?
["do>*__think_'s__think_*", 'is>*']

4898_10.q Do you think it's about patience durin